In [47]:
import pandas as pd
import numpy as np
import re
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

In [35]:
def clean_string(query):
    return re.sub('\s+', ' ', query.replace('(', ' ').replace(')', ' ').replace(',', ' , ')).strip()

In [36]:
df = pd.read_csv('../data/all_data.csv')
df['payload'] = df['payload'].apply(clean_string)

In [44]:
count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
raw_counts = count_vect.fit_transform(df['payload'])

# Display features
features = count_vect.get_feature_names()
print('Number of features: ' + str(len(features)))

In [51]:
# Append metadata to sparse matrix that will be used to train the classifier
num_feats = df[['length']].values
train_data = sparse.hstack((raw_counts, num_feats))